In [3]:
import pandas as pd

In [4]:
item_features = pd.read_parquet("features/item_features.parquet")

client_features = pd.read_parquet("features/client_features.parquet")

client_item_features = pd.read_parquet("features/client_item_features.parquet")

transaction_features = pd.read_parquet("features/transaction_features.parquet")

In [5]:
transaction_items__train = pd.read_csv(
    "data/transaction_items__train.csv",
    na_values="unknown",
    dtype={"transaction_id": "Int64", "item_id": "Int64", "add_to_cart_order": "Int64", "previous_bought": "Int64"},
)
transaction_items__train.drop(columns="Unnamed: 0", inplace=True)
transaction_items__train.dropna(subset=["transaction_id"], inplace=True)
transaction_items__train.set_index("transaction_id", inplace=True)

In [6]:
transaction_items__prior = pd.read_csv(
    "data/transaction_items__prior.csv",
    na_values="unknown",
    dtype={"transaction_id": "Int64", "item_id": "Int64", "add_to_cart_order": "Int64", "previous_bought": "Int64"},
)

transaction_items__prior.drop(columns="Unnamed: 0", inplace=True)
transaction_items__prior.dropna(subset=["transaction_id"], inplace=True)
transaction_items__prior.set_index("transaction_id", inplace=True)

In [7]:
transactions = pd.read_csv(
    "data/transactions.csv",
    na_values="unknown",
    dtype={
        "transaction_id": "Int64",
        "customer_id": "Int64",
        "transaction_number": "Int64",
        "day_of_week": "Int64",
        "time_of_day": "Int64",
        "days_since_prior_order": "Int64",
    },
)

transactions.drop(columns="Unnamed: 0", inplace=True)
transactions.dropna(subset=["transaction_id"], inplace=True)
transactions.set_index("transaction_id", inplace=True)

In [8]:
df_prior = pd.merge(transaction_items__prior, transactions, on="transaction_id")
df_train = pd.merge(transaction_items__train, transactions, on="transaction_id")

In [35]:
train_data = pd.DataFrame(index=df_prior.groupby(["customer_id", "item_id"]).size().index)
train_data["y"] = df_train.groupby(["customer_id", "item_id"]).agg({"item_id": "count"})
train_data["y"] = train_data["y"].fillna(0)

In [36]:
train_data

y
customer_id item_id     
1           196      1.0
            10258    1.0
            10326    0.0
            12427    0.0
            13032    1.0
...                  ...
206209      43961    0.0
            44325    0.0
            48370    0.0
            48697    0.0
            48742    0.0

[12140563 rows x 1 columns]

In [37]:
train_data = train_data.join(item_features, on="item_id", rsuffix="__item")

In [38]:
train_data = train_data.join(client_features, on="customer_id", rsuffix="__client")

In [39]:
train_data = train_data.join(client_item_features, on=["customer_id", "item_id"], rsuffix="__client_item")

In [40]:
customer_transaction = (
    df_train[["customer_id"]].reset_index().groupby(["customer_id", "transaction_id"]).count().reset_index(1)
)
train_data = train_data.join(customer_transaction)

In [41]:
train_data = train_data.join(transaction_features, on="transaction_id", rsuffix="__transaction")

In [43]:
train_data.dropna().to_parquet("sets/train_data.parquet")